In [1]:
import os
from dotenv import load_dotenv
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

load_dotenv()

USERNAME = os.getenv('USERNAME')
SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
SPOTIPY_REDIRECT_URI = os.getenv('SPOTIPY_REDIRECT_URI')

token = util.prompt_for_user_token(USERNAME, 'user-library-read')

spotify = spotipy.Spotify(
    auth=token, client_credentials_manager=SpotifyClientCredentials())

In [2]:
artist_uris = set()

offset = 0
saved_tracks = spotify.current_user_saved_tracks(limit=50, offset=offset)
while len(saved_tracks['items']) > 0:
    for track in saved_tracks['items']:
        for artist in track['track']['artists']:
            if artist['uri'] not in artist_uris:
                artist_uris.add(artist['uri'])
    offset += 50
    saved_tracks = spotify.current_user_saved_tracks(limit=50, offset=offset)

In [3]:
from tqdm.notebook import tqdm

artist_dict = {}
frequency_dict = {}

for artist_uri in tqdm(artist_uris):
    artist_dict[artist_uri] = spotify.artist(artist_uri)
    del artist_dict[artist_uri]['uri']
    del artist_dict[artist_uri]['type']
    followers = artist_dict[artist_uri]['followers']['total']
    del artist_dict[artist_uri]['followers']['total']
    artist_dict[artist_uri]['followers'] = followers
    url = artist_dict[artist_uri]['external_urls']['spotify']
    del artist_dict[artist_uri]['external_urls']
    artist_dict[artist_uri]['url'] = url
    artist_dict[artist_uri]['genres'] = set(artist_dict[artist_uri][
        'genres'])
    artist_dict[artist_uri]['images'] = {
        image['url'] for image in artist_dict[artist_uri]['images']}

    if artist_uri not in frequency_dict:
        frequency_dict[artist_uri] = 0

    results = spotify.artist_related_artists(artist_uri)
    if len(results['artists']) == 20:
        artist_dict[artist_uri]['related_artists'] = set()
        index = 0
        for result in results['artists']:
            related_artist_uri = result['uri']
            artist_dict[artist_uri]['related_artists'].add(
                related_artist_uri)
            if related_artist_uri not in artist_dict:
                artist_dict[related_artist_uri] = result
                del artist_dict[related_artist_uri]['uri']
                del artist_dict[related_artist_uri]['type']
                followers = artist_dict[related_artist_uri]['followers'][
                    'total']
                del artist_dict[related_artist_uri]['followers']['total']
                artist_dict[related_artist_uri]['followers'] = followers
                url = artist_dict[related_artist_uri]['external_urls'][
                    'spotify']
                del artist_dict[related_artist_uri]['external_urls']
                artist_dict[related_artist_uri]['url'] = url
                artist_dict[related_artist_uri]['genres'] = set(
                    artist_dict[related_artist_uri]['genres'])
                artist_dict[artist_uri]['images'] = {
                    image['url'] for image in artist_dict[
                        related_artist_uri]['images']}
                
            if related_artist_uri not in frequency_dict:
                frequency_dict[related_artist_uri] = 0
            frequency_dict[related_artist_uri] += 1
    
uris_to_delete = {artist for artist in frequency_dict if (frequency_dict[
    artist] < 2) and (artist not in artist_uris)}

artist_dict = {artist_uri: artist_dict[artist_uri] for artist_uri in 
               artist_dict if artist_uri not in uris_to_delete}

for artist_uri in artist_dict:
    if 'related_artists' in artist_dict[artist_uri]:
        artist_dict[artist_uri]['related_artists'] = artist_dict[artist_uri][
            'related_artists'] - uris_to_delete
    for key in artist_dict[artist_uri]:
        if type(artist_dict[artist_uri][key]) == set:
            artist_dict[artist_uri][key] = list(artist_dict[artist_uri][key])

In [4]:
import json

with open('artist_dict.json', 'w') as file:
    json.dump(artist_dict, file)